<a href="https://colab.research.google.com/github/agnair00/playdate_with_python/blob/main/mnist_basics_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastbook;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from fastai.vision.all import *
from fastbook import *
import torch

In [3]:
path = untar_data(URLs.MNIST_SAMPLE); path

Path('/root/.fastai/data/mnist_sample')

First step is create our dataloaders. Atthe end of it, we should have our training dataloader and our validation dataloader.

In [4]:
train_3_stack = torch.stack([tensor(Image.open(p)).float() for p in (path/'train/3').ls()])
train_3_stack.shape

torch.Size([6131, 28, 28])

In [5]:
train_7_stack = torch.stack([tensor(Image.open(p)).float() for p in (path/'train/7').ls()])
train_7_stack.shape

torch.Size([6265, 28, 28])

In [6]:
train_x = torch.cat([train_3_stack, train_7_stack]).flatten(1)
train_x.shape

torch.Size([12396, 784])

In [7]:
train_y = tensor([1]*len(train_3_stack) + [0]*len(train_7_stack)).float().unsqueeze(1)
train_y.shape

torch.Size([12396, 1])

In [8]:
valid_3_stack = torch.stack([tensor(Image.open(p)).float() for p in (path/'valid/3').ls()])
valid_3_stack.shape

torch.Size([1010, 28, 28])

In [9]:
valid_7_stack = torch.stack([tensor(Image.open(p)).float() for p in (path/'valid/7').ls()])
valid_7_stack.shape

torch.Size([1028, 28, 28])

In [10]:
valid_x = torch.cat([valid_3_stack, valid_7_stack]).flatten(1)
valid_x.shape

torch.Size([2038, 784])

In [11]:
valid_y = tensor([1]*len(valid_3_stack) + [0]*len(valid_7_stack)).float().unsqueeze(1)
valid_y.shape

torch.Size([2038, 1])

In [12]:
train_dset = list(zip(train_x, train_y))

In [13]:
train_dl = torch.utils.data.DataLoader(train_dset, batch_size=256)
len(train_dl)

49

So our training dataloader has 49 batches

In [14]:
valid_dset = list(zip(valid_x, valid_y))
valid_dl = torch.utils.data.DataLoader(valid_dset, batch_size=256)
len(valid_dl)

8

So now we have our training and our validation dataloaders

In [15]:
def init_params(size):
    return torch.randn(size).requires_grad_()

In [16]:
params = init_params((784, 1)), init_params(1)

In [17]:
def linear1(xb, params):
    weights, bias = params
    return (xb @ weights) + bias

In [18]:
def mnist_loss(predictions, targets):
    predictions = predictions.sigmoid()
    return torch.where(targets == 1.0, 1 - predictions, predictions).mean()

In [19]:
mnist_loss(linear1(train_x[:4], params), train_y[:4])

tensor(1., grad_fn=<MeanBackward0>)

In [104]:
def calc_grad1(model, loss_f, params, xb, yb):
    preds = model(xb, params)
    loss = loss_f(preds, yb)
    print(loss)
    loss.backward()

In [21]:
for param in params:
    try:
        print(param.grad.shape)
    except:
        print('param has no grad')

param has no grad
param has no grad


In [22]:
calc_grad1(linear1, mnist_loss, params, train_x[:4], train_y[:4])

In [23]:
for param in params:
    try:
        print(param.grad.shape)
    except:
        print('param has no grad')

torch.Size([784, 1])
torch.Size([1])


In [24]:
def train_epoch1(train_dl, model, loss, params, lr):
    for xb, yb in train_dl:
        calc_grad1(model, loss, params, xb, yb)
        for param in params:
            with torch.no_grad():
                param -= lr * param.grad
            param.grad.zero_()

In [25]:
def batch_accuracy1(preds, yb):
    preds = preds.sigmoid()
    return ((preds > 0.5) == yb).float().mean()

In [26]:
def validate_epoch1(valid_dl, model, params):
    with torch.no_grad():
        accs = [batch_accuracy1(model(xb, params), yb) for xb, yb in valid_dl]
        accuracy = torch.stack(accs)
    return round(accuracy.mean().item(), 4)

In [27]:
validate_epoch1(valid_dl, linear1, params)

0.5451

In [105]:
lr = 1.0
for _ in range(1):
    train_epoch1(train_dl, linear1, mnist_loss, params, lr)
    print(validate_epoch1(valid_dl, linear1, params), end=' ')

tensor(0.1094, grad_fn=<MeanBackward0>)
tensor(0.0547, grad_fn=<MeanBackward0>)
tensor(0.0664, grad_fn=<MeanBackward0>)
tensor(0.1133, grad_fn=<MeanBackward0>)
tensor(0.0742, grad_fn=<MeanBackward0>)
tensor(0.0859, grad_fn=<MeanBackward0>)
tensor(0.0938, grad_fn=<MeanBackward0>)
tensor(0.0977, grad_fn=<MeanBackward0>)
tensor(0.0859, grad_fn=<MeanBackward0>)
tensor(0.0898, grad_fn=<MeanBackward0>)
tensor(0.0820, grad_fn=<MeanBackward0>)
tensor(0.0674, grad_fn=<MeanBackward0>)
tensor(0.0234, grad_fn=<MeanBackward0>)
tensor(0.0352, grad_fn=<MeanBackward0>)
tensor(0.0117, grad_fn=<MeanBackward0>)
tensor(0.0312, grad_fn=<MeanBackward0>)
tensor(0.0156, grad_fn=<MeanBackward0>)
tensor(0.0156, grad_fn=<MeanBackward0>)
tensor(0.0469, grad_fn=<MeanBackward0>)
tensor(0.0234, grad_fn=<MeanBackward0>)
tensor(0.0234, grad_fn=<MeanBackward0>)
tensor(0.0391, grad_fn=<MeanBackward0>)
tensor(0.0312, grad_fn=<MeanBackward0>)
tensor(0.0352, grad_fn=<MeanBackward0>)
tensor(0.0195, grad_fn=<MeanBackward0>)


In [29]:
torch.nn.Linear(4, 1)(torch.arange(12).float().reshape(3,4))

tensor([[0.7588],
        [3.1500],
        [5.5412]], grad_fn=<AddmmBackward0>)

In [102]:
# Per batch
def calc_grad(xb, yb, model, loss_f):
    preds = model(xb)
    loss = loss_f(preds, yb)
    print(loss)
    loss.backward()

In [80]:
def train_epoch(train_dl, model, loss_f):
    for xb, yb in train_dl:
        calc_grad(xb, yb, model, loss_f)
        for p in model.parameters():
            with torch.no_grad():
                p -= 1.0 * p.grad
            p.grad.zero_()

In [81]:
def batch_accuracy(predictions, yb):
    predictions = predictions.sigmoid()
    return ((predictions > 0.5) == yb).float().mean()

In [82]:
def validate_epoch(valid_dl, model):
    accs = [batch_accuracy(model(xb), yb) for xb, yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)

In [86]:
def train_model(train_dl, valid_dl, model, loss_f, epochs):
    for _ in range(epochs):
        train_epoch(train_dl, model, loss_f)
        #print(validate_epoch(valid_dl, model), end= ' ')

In [87]:
linear_model = torch.nn.Linear(28*28, 1)
linear_model

Linear(in_features=784, out_features=1, bias=True)

In [106]:
train_model(train_dl, valid_dl, linear_model, mnist_loss, 2)

tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
tensor(0.0508, grad_fn=<MeanBackward0>)
tensor(1., grad_fn=<MeanBackward0>)
tensor(1., grad_fn=<MeanBackward0>)
tensor(1., grad_fn=<MeanBackward0>)
tensor(1., grad_fn=<Mean

In [89]:
with torch.no_grad():
   print(validate_epoch(valid_dl, linear_model))

0.4932


In [90]:
linear_model(valid_x[1500]).sigmoid()

tensor([1.], grad_fn=<SigmoidBackward0>)